In [3]:
import asyncio
import os
from pathlib import Path
from playwright.async_api import async_playwright

async def test_video_recording_properly():
    """Test video recording with proper settings and verification"""
    
    print("=== Testing Video Recording ===")
    
    output_dir = Path("output/screencasts/video_test")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    playwright = None
    browser = None
    context = None
    
    try:
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless=True)
        
        print(f"1. Output directory: {output_dir}")
        print(f"2. Directory exists: {output_dir.exists()}")
        
        # Create context with video recording
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            record_video_dir=str(output_dir),
            record_video_size={'width': 1920, 'height': 1080}
        )
        
        page = await context.new_page()
        
        # Do some actual interactions to create video content
        print("3. Creating video content...")
        await page.goto('https://srajappa.github.io/')
        await page.wait_for_timeout(2000)
        
        await page.goto('https://httpbin.org/html')
        await page.wait_for_timeout(2000)
        
        # Scroll to create movement
        await page.evaluate("window.scrollTo(0, 500)")
        await page.wait_for_timeout(1000)
        
        await page.evaluate("window.scrollTo(0, 0)")
        await page.wait_for_timeout(1000)
        
        print("4. Closing context to finalize video...")
        await context.close()
        context = None
        
        # Wait a moment for file system to sync
        await asyncio.sleep(2)
        
        print("5. Checking for video files...")
        video_files = list(output_dir.glob("*.webm"))
        mp4_files = list(output_dir.glob("*.mp4"))
        all_files = list(output_dir.iterdir())
        
        print(f"   WebM files: {len(video_files)}")
        print(f"   MP4 files: {len(mp4_files)}")
        print(f"   All files in directory:")
        for file in all_files:
            size = file.stat().st_size if file.is_file() else 0
            print(f"     - {file.name} ({size} bytes)")
        
        if video_files:
            video_file = video_files[0]
            size = video_file.stat().st_size
            print(f"\n✅ Video recorded successfully!")
            print(f"   File: {video_file}")
            print(f"   Size: {size} bytes")
            return True
        else:
            print("\n❌ No video files found")
            return False
            
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return False
        
    finally:
        if context:
            await context.close()
        if browser:
            await browser.close()
        if playwright:
            await playwright.stop()

# Test it
result = await test_video_recording_properly()
print(f"\nFinal result: {result}")

=== Testing Video Recording ===
1. Output directory: output/screencasts/video_test
2. Directory exists: True
3. Creating video content...
4. Closing context to finalize video...
5. Checking for video files...
   WebM files: 2
   MP4 files: 0
   All files in directory:
     - 386e684a9cd50855e79159bbb722804d.webm (882855 bytes)
     - 49d01eb8de19ac093449328c27b22de6.webm (635960 bytes)

✅ Video recorded successfully!
   File: output/screencasts/video_test/386e684a9cd50855e79159bbb722804d.webm
   Size: 882855 bytes

Final result: True
